In [1]:
# External dependencies
from mecode import GMatrix


In [2]:
# Local denpendencies
from src.helpers.gmatrix_extensions import get_gmatrix, pen_down, pen_up
from src.repeating_patterns.draw_unit import draw_unit
from src.repeating_patterns.draw_repeating_pattern import draw_repeating_pattern


# Patterns


In [3]:
square_pattern = [
    [(0, 10), (0, 0), (10, 0)]  # The left and top sides of the square
]  # This pattern contains only one stroke.

rotated_square_pattern = [
    [(0, 0), (20, 20)],  # diagonal top-left to bottom-right
    [(20, 15), (15, 10), (20, 5)],  # right triangle
    [(20, 0), (0, 20)],  # diagonal top-right to bottom-left
    [(0, 15), (5, 10), (0, 5)],  # left triangle
    [(5, 0), (10, 5), (15, 0)],  # top triangle
    [(5, 20), (10, 15), (15, 20)],  # bottom triangle
]


# Test `draw_unit`


In [4]:
# Unit test for draw_unit
g: GMatrix = get_gmatrix("output/test_pattern.gcode")
pen_up(g)
draw_unit(g, rotated_square_pattern)


# Draw repeating pattern

In [6]:
# Useful GCode snippets
end_code = ""

# Initialise GCode generation and write start code
g: GMatrix = get_gmatrix("output/repeating_pattern_rotated_square.gcode")

# Move the pen to a reasonable starting position
pen_up(g)
g.move(30, 30, rapid=True)

# Draw the pattern
draw_repeating_pattern(g, 20, 20, rotated_square_pattern, 1, (4, 5))

# Write end code
g.pop_matrix()
g.write(end_code)
